# `pytket-mbqc` Simple Example
Here we present some simple use cases of `pytket-mbqc`.

## X Gate
The X can be implemented in brickwork style MBQC as the sequence H-Rz(pi)-H = J(pi)-H. This requires a three qubit cluster state entangle in a line. This is constructed by initialising an input qubit, a graph vertex and an output vertex. These are then connected in a line and measured together at the end.

In [ ]:
import networkx as nx
from pytket.circuit.display import render_circuit_jupyter
from pytket.unit_id import BitRegister

from pytket_mbqc_py import GraphCircuit

circuit = GraphCircuit(n_qubits_total=3)

input_qubit, index_one = circuit.add_input_vertex()

circuit.add_graph_vertex()
circuit.add_edge(vertex_one=0, vertex_two=1)

circuit.add_output_vertex()
circuit.add_edge(vertex_one=1, vertex_two=2)

circuit.corrected_measure(vertex=0, t_multiple=0)
circuit.corrected_measure(vertex=1, t_multiple=4)
circuit._apply_correction(vertex=2)

output_reg = BitRegister(name='output', size=1)
circuit.add_c_register(register=output_reg)
circuit.Measure(qubit=circuit.output_qubits[2], bit=output_reg[0])

print(*circuit.get_commands(), sep='\n')
nx.draw(circuit.graph, with_labels=True)

Let us run the resulting circuit. We should recover the 1 state 100% of the time as we are applying an X gate to the 0 state.

In [ ]:
from pytket.extensions.quantinuum import QuantinuumAPIOffline, QuantinuumBackend

from pytket_mbqc_py import get_wasm_file_handler

backend = QuantinuumBackend(
    device_name="H1-1LE",
    api_handler=QuantinuumAPIOffline(),
)

compiled_circuit = backend.get_compiled_circuit(circuit)
result = backend.run_circuit(
    circuit=compiled_circuit,
    n_shots=100,
    wasm_file_handler=get_wasm_file_handler(),
)

result.get_counts(output_reg)

## CX Gate
The CX gate is more complicated. In this case the gate can be decomposed into brickwork MBQC native operations as H_2 - CZ - H_1 - H-2 H_1 - H_2 - H_2 where unserscore gives the qubit the operation acts on. Recall that CZ - H_1 - H_2 is the native two qubit gate in brickwork MBQC. We can construct this circuit as follows.

Note that in this case qubits can be measured before the whole graph state has been constructed. This allows for some qubits to be reused in this case.

In [ ]:
input_state = (1,0)

circuit = GraphCircuit(n_qubits_total=5)

target_qubit, _ = circuit.add_input_vertex()
if input_state[1]:
    circuit.X(target_qubit)

circuit.add_graph_vertex()
circuit.add_edge(vertex_one=0, vertex_two=1)

control_qubit, _ = circuit.add_input_vertex()
if input_state[0]:
    circuit.X(control_qubit)

circuit.add_graph_vertex()
circuit.add_edge(vertex_one=1, vertex_two=3)
circuit.corrected_measure(vertex=0, t_multiple=0)

circuit.add_graph_vertex()
circuit.add_edge(vertex_one=2, vertex_two=4)

circuit.add_graph_vertex()
circuit.add_edge(vertex_one=3, vertex_two=5)
circuit.corrected_measure(vertex=1, t_multiple=0)

circuit.add_output_vertex()
circuit.add_edge(vertex_one=4, vertex_two=6)
circuit.corrected_measure(vertex=2, t_multiple=0)

circuit.add_output_vertex()
circuit.add_edge(vertex_one=5, vertex_two=7)

circuit.add_edge(vertex_one=5, vertex_two=6)

circuit.corrected_measure(vertex=3, t_multiple=0)
circuit.corrected_measure(vertex=4, t_multiple=0)
circuit.corrected_measure(vertex=5, t_multiple=0)

circuit.correct_outputs()
output_reg = BitRegister(name='output', size=2)
circuit.add_c_register(register=output_reg)
circuit.Measure(qubit=circuit.output_qubits[6], bit=output_reg[0])
circuit.Measure(qubit=circuit.output_qubits[7], bit=output_reg[1])

nx.draw(circuit.graph, with_labels=True)

Having initialised the |10> state above we expect to measure |11> 100% of the time, as we indeed do.

In [ ]:
compiled_circuit = backend.get_compiled_circuit(circuit)

n_shots = 100
result = backend.run_circuit(
    circuit=compiled_circuit,
    n_shots=n_shots,
    wasm_file_handler=get_wasm_file_handler(),
)

result.get_counts(output_reg)